In [1]:
from querymodule import fume_query, query_to_list, total_energy, total_time_sash_open
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
from dateutil import tz
import requests
import json
import matplotlib.pyplot as plt
import math
import scipy.stats as st

from dateutil import parser
import matplotlib.dates as mdates

points = pd.read_csv("pointnames.csv")

In [9]:
"""
Coversion factors:
From-Unit	To-Unit	factor
=========   =======     ========
Ton-Hour    $           0.198
kWh         $           0.094
Klbs        $           29.9
BTU         $           2.65E-05
Ton-Hour    lbCO2       0.275
kWh         lbCO2       0.439
Klbs        lbCO2       45
BTU         lbCO2       3.99E-05
"""

sashpoints = points["Sash"]
hoodpoints = points["Hood"]
occpoints = points["Occupancy"]
internaltemp = points["Internal Temp"]
externaltemp = points["External Temp"]

server  = points["Server"][0]
time_start = str(datetime(2022, 1,1))
time_end = str(datetime(2022, 12,31))
sash_points = points["Sash"]

In [10]:
#total_energy(cfm_point, sash_point, occ_point, internal_temp_point, external_temp_point, server, start, end, is_occupied)

energy_list = [total_energy(hoodpoints[i], sashpoints[i],occpoints[i],internaltemp[i],externaltemp[i],server,time_start,time_end,True) for i in range(len(sashpoints))]

In [11]:
mean_btu = [np.sum(i) for i in energy_list]
data = {"Name": points["Lab"], "BTU":mean_btu}
ndf = pd.DataFrame(data)
ndf["Cost"] = ndf["BTU"]*0.0000265
ndf["CO2"] = ndf["BTU"]* 0.0000399
ndf

,Name,BTU,Cost,CO2
0,g54,6.713141e+06,177.898229,267.854315
1,141,1.348969e+07,357.476843,538.238718
2,143,1.401541e+07,371.408322,559.214794
3,147b,1.352986e+07,358.541214,539.841299
4,153,1.072199e+07,284.132804,427.807505
...,...,...,...,...
61,445b,2.238213e+07,593.126518,893.047097
62,453,1.575869e+07,417.605304,628.771760
63,453b,1.695292e+07,449.252303,676.421392
64,457,2.057153e+07,545.145512,820.803998


In [12]:
total_cost = np.sum(ndf["Cost"])
total_cost

27867.169582410857

In [13]:
total_co2 = np.sum(ndf["CO2"])
total_co2

41958.49306936578